In [1]:
import pandas as pd

from aidream_registration import constants
from aidream_registration.dataloaders import AtlasImagingNiftiLoader
from aidream_registration.utils.cohort_utils import get_perfusion_patients
from sklearn.utils import resample

from torch.optim.lr_scheduler import ReduceLROnPlateau

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.model_selection import train_test_split


import numpy as np
import ants


In [2]:
torch.cuda.empty_cache()

In [3]:
list_patients = get_perfusion_patients()
print("Number of patients:", len(list_patients))


Number of patients: 186


In [4]:
# Load the atlas imaging nifti loader :
atlas_loader = AtlasImagingNiftiLoader(source_mri="PIPELINE_SS")


In [5]:
# First, load the hammersmith atlas, and create a mask :
path_hammersmith = constants.DIR_DATA / "hammersmith" / "T1w_ICBM_skullstripped.nii.gz"
ants_hammersmith = ants.image_read(str(path_hammersmith))

mask_hammersmith = ants_hammersmith > 0
print(f"Hammersmith mask volume: {mask_hammersmith.sum() / 1e3:.2f} cm3")


Hammersmith mask volume: 1886.57 cm3


In [6]:
# Load all segmentations in a dictionary :
dict_segmentations = {
    path_seg.name.removesuffix(".nii.gz"): ants.image_read(str(path_seg)) > 0
    for path_seg in (constants.DIR_DATA / "hammersmith").glob("*.nii.gz")
    if path_seg.name != "T1w_ICBM_skullstripped.nii.gz"
}

print("Number of segmentations:", len(dict_segmentations))

for segmentation, mask_segmentation in dict_segmentations.items():
    print(fr"{segmentation} volume: {mask_segmentation.sum() / 1e3:.2f} cm3")


Number of segmentations: 21
L.Ventricles volume: 6.72 cm3
R_insula volume: 16.48 cm3
R.Ventricles volume: 6.11 cm3
L.Temporal_lobe volume: 115.72 cm3
R.Occipital_lobe volume: 76.46 cm3
L.Grey_nuclei volume: 18.22 cm3
R.Grey_nuclei volume: 18.20 cm3
R.Limbic_lobe volume: 24.28 cm3
Brainstem volume: 26.94 cm3
L.frontal_lobe volume: 225.46 cm3
third_ventricle volume: 0.63 cm3
L.Post_fossea volume: 90.63 cm3
L.Parietal_lobe volume: 142.35 cm3
Corpus_callosum volume: 21.05 cm3
R.Temporal_lobe volume: 121.76 cm3
L.Occipital_lobe volume: 75.31 cm3
R.frontal_lobe volume: 227.50 cm3
R.Post_fossea volume: 88.57 cm3
R.Parietal_lobe volume: 141.59 cm3
L.Limbic_lobe volume: 24.96 cm3
L_insula volume: 16.62 cm3


In [7]:
# Create a custom ventricles mask by combining the lateral ventricles and the third ventricle :
mask_ventricles = dict_segmentations["L.Ventricles"] + dict_segmentations["R.Ventricles"] + dict_segmentations["third_ventricle"]
mask_ventricles = mask_ventricles > 0

print(f"Ventricles volume: {mask_ventricles.sum() / 1e3:.2f} cm3")

path_ventricles = constants.DIR_DATA / "hammersmith" / "custom" / "total_ventricles" / "ventricles.nii.gz"
path_ventricles.parent.mkdir(parents=True, exist_ok=True)
mask_ventricles.to_file(str(path_ventricles))


Ventricles volume: 13.47 cm3


In [8]:
# Create a mask for the neighbors of the ventricles :
dict_ventricles_neighbors = {}

for seg in ["L.Ventricles", "R.Ventricles", "third_ventricle"]:

    # Load the segmentation mask (prior) :
    mask_segmentation = dict_segmentations[seg]
    seg_idx = np.array(np.nonzero(mask_segmentation.numpy()))
    # Bounding box :
    bbox_min, bbox_max = np.min(seg_idx, axis=1), np.max(seg_idx, axis=1)
    # Dilate by 3 voxels :
    bbox_min, bbox_max = bbox_min - 3, bbox_max + 3
    np_bbox = np.zeros(mask_segmentation.shape)
    np_bbox[bbox_min[0]: bbox_max[0], bbox_min[1]: bbox_max[1], bbox_min[2]: bbox_max[2]] = 1

    dict_ventricles_neighbors[seg] = (ants.from_numpy(np_bbox, origin=mask_segmentation.origin, spacing=mask_segmentation.spacing, direction=mask_segmentation.direction) > 0) * mask_hammersmith

# Create mask_ventricles_neighborhood :
mask_ventricles_neighbors = dict_ventricles_neighbors["L.Ventricles"] + dict_ventricles_neighbors["R.Ventricles"] + dict_ventricles_neighbors["third_ventricle"]
mask_ventricles_neighbors = mask_ventricles_neighbors > 0

print(f"Ventricles neighborhood volume: {mask_ventricles_neighbors.sum() / 1e3:.2f} cm3")

# Create mask for brain without the ventricles' neighborhood :
mask_no_ventricles_neighbors = ants_hammersmith * (1 - mask_ventricles_neighbors)
mask_no_ventricles_neighbors = mask_no_ventricles_neighbors > 0

print(f"Brain without ventricles neighborhood volume: {mask_no_ventricles_neighbors.sum() / 1e3:.2f} cm3")


Ventricles neighborhood volume: 440.61 cm3
Brain without ventricles neighborhood volume: 1445.96 cm3


In [9]:
# Create a CSF segmentation for the voxels outside the ventricles that are not segmented :
mask_csf_no_ventricles_neighbors = mask_no_ventricles_neighbors

for seg, mask_segmentation in dict_segmentations.items():

    if seg not in ["L.Ventricles", "R.Ventricles", "third_ventricle"]:
        mask_csf_no_ventricles_neighbors = mask_csf_no_ventricles_neighbors * (1 - mask_segmentation)

assert set(np.unique(mask_csf_no_ventricles_neighbors.numpy())) == {0, 1}

print(f"CSF volume outside of the ventricles neighborhood: {mask_csf_no_ventricles_neighbors.sum() / 1e3:.2f} cm3")

# Create a mask for the segmentations excluding the ventricles and the CSF :
mask_no_csf_no_ventricles_neighbors = mask_no_ventricles_neighbors * (1 - mask_csf_no_ventricles_neighbors)

print(f"Brain without ventricles and CSF volume: {mask_no_csf_no_ventricles_neighbors.sum() / 1e3:.2f} cm3")


CSF volume outside of the ventricles neighborhood: 259.03 cm3
Brain without ventricles and CSF volume: 1186.93 cm3


In [10]:
# Generate spatial coordinate images :
x,y,z = ants_hammersmith.shape

x_coords = np.linspace(0, 1, x)[:, None, None]
y_coords = np.linspace(0, 1, y)[None, :, None]
z_coords = np.linspace(0, 1, z)[None, None, :]

x_map = np.broadcast_to(x_coords, (x, y, z))
y_map = np.broadcast_to(y_coords, (x, y, z))
z_map = np.broadcast_to(z_coords, (x, y, z))

ants_x_map = ants.from_numpy(x_map, origin=ants_hammersmith.origin, spacing=ants_hammersmith.spacing, direction=ants_hammersmith.direction)
ants_y_map = ants.from_numpy(y_map, origin=ants_hammersmith.origin, spacing=ants_hammersmith.spacing, direction=ants_hammersmith.direction)
ants_z_map = ants.from_numpy(z_map, origin=ants_hammersmith.origin, spacing=ants_hammersmith.spacing, direction=ants_hammersmith.direction)



In [15]:
radius = 2
beta = 0.3

mrf = f"[{beta}, {radius}x{radius}x{radius}]"

print(fr"Applying Atropos with MRF = {mrf} ...")

atropos_results = ants.atropos(a=[ants_hammersmith],
                               x=mask_hammersmith,
                               m=mrf,
                               c="[10,0]",
                               i=[mask_ventricles, mask_csf_no_ventricles_neighbors, mask_no_csf_no_ventricles_neighbors],
                               verbose=1)

# ants_ventricles_atropos = (atropos_results["segmentation"] == 1) * mask_ventricles_neighbors
ants_ventricles_atropos = atropos_results["segmentation"]

path_ventricles_atropos = constants.DIR_DATA / "hammersmith" / "custom" / f"ventricle_atropos_{mrf}.nii.gz"
path_ventricles_atropos.parent.mkdir(parents=True, exist_ok=True)
ants_ventricles_atropos.to_file(str(path_ventricles_atropos))


Applying Atropos with MRF = [0.3, 2x2x2] ...

Running Atropos for 3-dimensional images.

Progress: 
  Iteration 0 (of 10): posterior probability = 0 (annealing temperature = 1)
  Iteration 1 (of 10): posterior probability = 0.980294 (annealing temperature = 1)
  Iteration 2 (of 10): posterior probability = 0.992723 (annealing temperature = 1)
  Iteration 3 (of 10): posterior probability = 0.993611 (annealing temperature = 1)
  Iteration 4 (of 10): posterior probability = 0.99427 (annealing temperature = 1)
  Iteration 5 (of 10): posterior probability = 0.994788 (annealing temperature = 1)
  Iteration 6 (of 10): posterior probability = 0.99518 (annealing temperature = 1)
  Iteration 7 (of 10): posterior probability = 0.995501 (annealing temperature = 1)
  Iteration 8 (of 10): posterior probability = 0.995698 (annealing temperature = 1)
  Iteration 9 (of 10): posterior probability = 0.995889 (annealing temperature = 1)

Writing output:
  Writing posterior image (class 1)
  Writing poster

In [16]:
# Apply atropos to improve the quality fo the ventricles segmentation using the ventricles mask as a prior :

radius = 2
beta = 0.3

mrf = f"[{beta}, {radius}x{radius}x{radius}]"

print(fr"Applying Atropos with MRF = {mrf} ...")

atropos_results = ants.atropos(a=[ants_hammersmith, ants_x_map, ants_y_map],
                               x=mask_hammersmith,
                               m=mrf,
                               c="[10,0]",
                               i=[mask_ventricles, mask_csf_no_ventricles_neighbors, mask_no_csf_no_ventricles_neighbors],
                               verbose=1)

# ants_ventricles_atropos = (atropos_results["segmentation"] == 1) * mask_ventricles_neighbors
ants_ventricles_atropos = atropos_results["segmentation"]

path_ventricles_atropos = constants.DIR_DATA / "hammersmith" / "custom" / f"ventricle_atropos_{mrf}_coords.nii.gz"
path_ventricles_atropos.parent.mkdir(parents=True, exist_ok=True)
ants_ventricles_atropos.to_file(str(path_ventricles_atropos))

Applying Atropos with MRF = [0.3, 2x2x2] ...


In [13]:
# Now, let's treat the problem as a Machine Learning classification problem :
# The training data will be the ventricles' segmentation and the voxels outside the ventricles' neighborhood.
# The prediction will be done on the voxels inside the ventricles' neighborhood, that are not segmented as ventricles.

# First, create the training data :
# Add the ventricles' segmentation :
ventricles_idx = np.array(np.nonzero(mask_ventricles.numpy()))
ventricles_intensity = ants_hammersmith.numpy()[ventricles_idx[0], ventricles_idx[1], ventricles_idx[2]]
df_ventricles = pd.DataFrame({"x": ventricles_idx[0], "y": ventricles_idx[1], "z": ventricles_idx[2],
                              "intensity": ventricles_intensity, "label": 0})

# now Add the voxels outside the ventricles' neighborhood that are CSF :
csf_no_ventricles_neighbors_idx = np.array(np.nonzero(mask_csf_no_ventricles_neighbors.numpy()))
csf_no_ventricles_neighbors_intensity = ants_hammersmith.numpy()[csf_no_ventricles_neighbors_idx[0], csf_no_ventricles_neighbors_idx[1], csf_no_ventricles_neighbors_idx[2]]
df_csf_no_ventricles_neighbors = pd.DataFrame({"x": csf_no_ventricles_neighbors_idx[0], "y": csf_no_ventricles_neighbors_idx[1], "z": csf_no_ventricles_neighbors_idx[2],
                                               "intensity": csf_no_ventricles_neighbors_intensity, "label": 1})

# Now, add the voxels outside the ventricles' neighborhood that are not CSF :
no_csf_no_ventricles_neighbors_idx = np.array(np.nonzero(mask_no_csf_no_ventricles_neighbors.numpy()))
no_csf_no_ventricles_neighbors_intensity = ants_hammersmith.numpy()[no_csf_no_ventricles_neighbors_idx[0], no_csf_no_ventricles_neighbors_idx[1], no_csf_no_ventricles_neighbors_idx[2]]
df_no_csf_no_ventricles_neighbors = pd.DataFrame({"x": no_csf_no_ventricles_neighbors_idx[0], "y": no_csf_no_ventricles_neighbors_idx[1], "z": no_csf_no_ventricles_neighbors_idx[2],
                                                  "intensity": no_csf_no_ventricles_neighbors_intensity, "label": 2})

# Concatenate the dataframes :
df_train = pd.concat([df_ventricles, df_csf_no_ventricles_neighbors, df_no_csf_no_ventricles_neighbors], ignore_index=True)
print(fr"len(df_train): {len(df_train)}")

X_train, y_train = df_train[["x", "y", "z", "intensity"]].values, df_train["label"].values

print(f"Number of training samples: {len(X_train)}")

print(f"percentage of ventricles in the training set: {np.mean(y_train == 0):.2f}")
print(f"percentage of CSF in the training set: {np.mean(y_train == 1):.2f}")
print(f"percentage of brain in the training set: {np.mean(y_train == 2):.2f}")


len(df_train): 1459429
Number of training samples: 1459429
percentage of ventricles in the training set: 0.01
percentage of CSF in the training set: 0.18
percentage of brain in the training set: 0.81


In [14]:
# Let's resample the training data to balance the classes :
ventricles = df_train[df_train["label"] == 0]
csf = df_train[df_train["label"] == 1]
brain = df_train[df_train["label"] == 2]

# the desired distribution is 2:2:6
desired_ventricles_size = desired_csf_size = int(len(df_train) * 0.2)
desired_brain_size = len(df_train) - desired_ventricles_size - desired_csf_size

# Oversampling the ventricles and the CSF :
ventricles_resampled = resample(ventricles, n_samples=desired_ventricles_size, replace=True, random_state=42)
csf_resampled = resample(csf, n_samples=desired_csf_size, replace=True, random_state=42)

# Downsampling the brain :
brain_resampled = resample(brain, n_samples=desired_brain_size, replace=False, random_state=42)

# Concatenate the resampled dataframes :
df_train_resampled = pd.concat([ventricles_resampled, csf_resampled, brain_resampled], ignore_index=True)

X_train, X_val, y_train, y_val = train_test_split(df_train_resampled[["x", "y", "z", "intensity"]].values, df_train_resampled["label"].values, test_size=0.2, random_state=42)

print(f"Number of training samples: {len(X_train)}")
print(f"Number of validation samples: {len(X_val)}")

print(f"percentage of ventricles in the training set: {np.mean(y_train == 0):.2f}, and in the validation set: {np.mean(y_val == 0):.2f}")
print(f"percentage of CSF in the training set: {np.mean(y_train == 1):.2f}, and in the validation set: {np.mean(y_val == 1):.2f}")
print(f"percentage of brain in the training set: {np.mean(y_train == 2):.2f}, and in the validation set: {np.mean(y_val == 2):.2f}")


Number of training samples: 1167543
Number of validation samples: 291886
percentage of ventricles in the training set: 0.20, and in the validation set: 0.20
percentage of CSF in the training set: 0.20, and in the validation set: 0.20
percentage of brain in the training set: 0.60, and in the validation set: 0.60


In [15]:
# Let's do the prediction set on the voxels inside the ventricles' neighborhood  that are not segmented as ventricles :
mask_pred = mask_ventricles_neighbors * (1 - mask_ventricles)

pred_idx = np.array(np.nonzero(mask_pred.numpy()))
pred_intensity = ants_hammersmith.numpy()[pred_idx[0], pred_idx[1], pred_idx[2]]
df_pred = pd.DataFrame({"x": pred_idx[0], "y": pred_idx[1], "z": pred_idx[2], "intensity": pred_intensity})

X_pred = df_pred[["x", "y", "z", "intensity"]].values
print(f"Nuber of prediction samples: {len(X_pred)} / {mask_ventricles_neighbors.sum()}")


Nuber of prediction samples: 427148 / 440611


In [16]:
# Build the neural network classifier :
class VoxelClassifier(nn.Module):
    def __init__(self):
        super(VoxelClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(4, 8),  # Input: 4 features, first hidden layer: 8 neurons
            nn.ReLU(),
            nn.Linear(8, 16),  # Second hidden layer: 16 neurons
            nn.ReLU(),
            nn.Linear(16, 32), # Third hidden layer: 33 neurons
            nn.ReLU(),
            nn.Linear(32, 64), # Fourth hidden layer: 64 neurons
            nn.ReLU(),
            nn.Linear(64, 32),  # Fifth hidden layer: 32 neurons
            nn.ReLU(),
            nn.Linear(32, 16),  # Sixth hidden layer: 16 neurons
            nn.ReLU(),
            nn.Linear(16, 3)    # Output: 3 classes
        )

    def forward(self, x):
        return self.model(x)


In [17]:
model = VoxelClassifier()

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Move model to GPU
model.to(device)


cuda


VoxelClassifier(
  (model): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=32, bias=True)
    (9): ReLU()
    (10): Linear(in_features=32, out_features=16, bias=True)
    (11): ReLU()
    (12): Linear(in_features=16, out_features=3, bias=True)
  )
)

In [18]:
# Convert your training data into PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)  # Features
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)    # Labels

# Convert your validation data into PyTorch tensors
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)  # Features
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)    # Labels


In [19]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

batch_size = 2000

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=100000, shuffle=False)

# Verify DataLoaders by iterating through one batch
for X_batch, y_batch in train_loader:
    print("Training Batch:")
    print("  Features Shape:", X_batch.shape)
    print("  Labels Shape:", y_batch.shape)
    print("  Label Counts:", torch.bincount(y_batch))
    break  # Check only the first batch

for X_batch, y_batch in val_loader:
    print("Validation Batch:")
    print("  Features Shape:", X_batch.shape)
    print("  Labels Shape:", y_batch.shape)
    print("  Label Counts:", torch.bincount(y_batch))
    break  # Check only the first batch


Training Batch:
  Features Shape: torch.Size([2000, 4])
  Labels Shape: torch.Size([2000])
  Label Counts: tensor([ 436,  391, 1173], device='cuda:0')
Validation Batch:
  Features Shape: torch.Size([100000, 4])
  Labels Shape: torch.Size([100000])
  Label Counts: tensor([19728, 19982, 60290], device='cuda:0')


In [20]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Training and validation loop
num_epochs = 10000
for epoch in range(num_epochs):

    print(fr"-------------------- Epoch", epoch+1, "--------------------")
    # Training phase
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()           # Zero gradients
        outputs = model(X_batch)        # Forward pass
        loss = criterion(outputs, y_batch)  # Compute loss
        loss.backward()                 # Backward pass
        optimizer.step()                # Update weights

        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item()

            # Accuracy calculation
            predicted = outputs.argmax(dim=1)
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

    val_loss /= len(val_loader)
    val_accuracy = correct / total

    # Log metrics
    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"  Training Loss: {train_loss:.4f}")
    print(f"  Validation Loss: {val_loss:.4f}")
    print(f"  Validation Accuracy: {val_accuracy:.4f}")

    # Optional: Log GPU memory usage
    print(f"  GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    print(f"  GPU Memory Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")


-------------------- Epoch 1 --------------------
Epoch 1/10000:
  Training Loss: 0.4909
  Validation Loss: 0.3956
  Validation Accuracy: 0.8425
  GPU Memory Allocated: 0.06 GB
  GPU Memory Reserved: 0.16 GB
-------------------- Epoch 2 --------------------
Epoch 2/10000:
  Training Loss: 0.3833
  Validation Loss: 0.3734
  Validation Accuracy: 0.8492
  GPU Memory Allocated: 0.06 GB
  GPU Memory Reserved: 0.16 GB
-------------------- Epoch 3 --------------------
Epoch 3/10000:
  Training Loss: 0.3659
  Validation Loss: 0.3594
  Validation Accuracy: 0.8519
  GPU Memory Allocated: 0.06 GB
  GPU Memory Reserved: 0.16 GB
-------------------- Epoch 4 --------------------
Epoch 4/10000:
  Training Loss: 0.3545
  Validation Loss: 0.3459
  Validation Accuracy: 0.8580
  GPU Memory Allocated: 0.06 GB
  GPU Memory Reserved: 0.16 GB
-------------------- Epoch 5 --------------------
Epoch 5/10000:
  Training Loss: 0.3460
  Validation Loss: 0.3399
  Validation Accuracy: 0.8606
  GPU Memory Allocated: